In [ ]:
# Data Cleaning 
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# src-Verzeichnis relativ vom aktuellen Notebook-Verzeichnis
sys.path.append(str(Path().resolve().parent / "src"))

from src.data_cleaning import (batch_clean_all_tsv,
                               mark_outliers,
                               visualize_outliers
                               )
from src.data_handling import find_tsv_files
from src.data_normalization import batch_normalize_tpm

# Normalisierung der Rohdaten bevor Outlier entfernt werden
#data1_folder = "../data/raw_data_bb/"
#output1_folder = "../data/pre_normalized_data_bb/"
#gff_folder = "../data/raw_data_bb"

#batch_normalize_tpm(data1_folder, output1_folder, gff_folder, visual=False)

# Konfiguration der Ein- und Ausgabeverzeichnisse (Bold-Binders Daten)
#data_folder = "../data/pre_normalized_data_bb/"
data_folder = "../data/raw_data_bb/"
output_folder = "../data/cleaned_data_bb/"

# Konfiguration der Ein- und Ausgabeverzeichnisse (Bold-Binders Daten)
# data_folder = "../data/raw_data_all/"
# output_folder = "../data/cleaned_data_all/"

batch_clean_all_tsv(data_folder, output_folder, method='iqr',
                    visual=True)  # der Parameter visual steuert die Ausgabe der Visualisierung

In [ ]:
# Normalization
from src.data_normalization import batch_normalize_tpm

data_folder = "../data/cleaned_data_bb/"
output_folder = "../data/normalized_data_bb/"
gff_folder = "../data/raw_data_bb"

# # Konfiguration der Ein- und Ausgabeverzeichnisse (Aller Daten)
# data_folder = "../data/cleaned_data_all/"
# output_folder = "../data/normalized_data_all/"
# gff_folder = "../data/raw_data_all"

batch_normalize_tpm(data_folder, output_folder, gff_folder, phage_only=True,
                    visual=True)  # der Parameter visual steuert die Ausgabe der Visualisierung

In [ ]:
# sample based PCA
from src.sample_pca import process_directory
# Ausführung
input_dir = "../data/normalized_data_bb"
# Benutzerdefinierte Ausreißer (optional)
# user_defined_outliers = ["20_R3", "12_R1"]
# user_defined_outliers = None #automatiziert
user_defined_outliers = []

file_name = "Wolfram"
process_directory(input_dir, user_outliers=user_defined_outliers, filename_filter=file_name)
# process_directory(input_dir, user_outliers=user_defined_outliers)

# US4-7 Labeling

In [ ]:
# Labeling
from src.labeling import process_all_files, labeling_pipeline_means
from src.visualization import (
    plot_label_distribution_comparison,
    plot_aggregated_distribution
)

# Konfiguration der Ein- und Ausgabeverzeichnisse (Bold-Binders Daten)
data_folder = "../data/normalized_data_bb/"
output_folder = "../data/gene_labeling_bb/"
time_means_folder="../data/time_means/"

# # Konfiguration der Ein- und Ausgabeverzeichnisse (Aller Daten)
# data_folder = "../data/normalized_data_all/"
# output_folder = "../data/gene_labeling_all/"

# Verarbeitung aller Dateien
label_counts_std, label_counts_paper, _ = process_all_files(data_folder, output_folder)
label_counts_means = labeling_pipeline_means(data_folder,time_means_folder, output_folder, "gene_labels_means.tsv")

# Erstelle Visualisierungen
fig1 = plot_label_distribution_comparison(label_counts_std, label_counts_paper, label_counts_means)

fig2 = plot_aggregated_distribution(label_counts_std, label_counts_paper, label_counts_means)

print(f"Alle Ergebnisse wurden im Verzeichnis '{output_folder}' gespeichert.")

US8 Genome Maps

In [ ]:
# Genome Maps
from src.genome_map import visualize_from_merged_tsv_by_geneid

merged_tsv_path = "../data/gene_labeling_bb/gene_labels_means.tsv"
gff_phage_dir = "../data/raw_data_bb/phage_gff_files"
output_genome_maps_dir = "../data/genome_maps"

visualize_from_merged_tsv_by_geneid(
    merged_tsv_path,
    gff_phage_dir,
    output_genome_maps_dir
)

US9 Feature Extraction

In [ ]:
from pathlib import Path
import sys

# src-Verzeichnis relativ vom aktuellen Notebook-Verzeichnis
sys.path.append(str(Path().resolve().parent / "src"))

from src.feature_extraction import extract_features

# Pfade definieren
gff_dir = "../data/raw_data_bb/phage_gff_files"
fasta_dir = "../data/raw_data_bb"
#Auswahl des Labeling Verfahrens
labels_file = "../data/gene_labeling_bb/gene_labels_means.tsv"
output_file = "../data/gene_features/gene_features.tsv"

# Stelle sicher, dass das Ausgabeverzeichnis existiert
Path(output_file).parent.mkdir(parents=True, exist_ok=True)

try:
    # Feature Extraktion ausführen
    results = extract_features(
        gff_directory=gff_dir,
        fasta_directory=fasta_dir,
        labels_file=labels_file,
        output_file=output_file
    )

    if results is not None and not results.empty:
        # Erste paar Zeilen anzeigen
        print("\nErste Einträge der extrahierten Daten:")
        display(results.head())

        # Statistiken anzeigen
        print("\nStatistiken:")
        print(f"Anzahl der extrahierten Gene: {len(results)}")
        print(f"Anzahl einzigartiger Phagen: {results['source_study'].nunique()}")
        print(f"Verteilung der Labels:\n{results['label'].value_counts()}")
    else:
        print("Keine Daten wurden extrahiert.")

except Exception as e:
    print(f"Fehler bei der Feature-Extraktion: {str(e)}")


In [ ]:
# Feature-Matrix erzeugen, laden und visualisieren
import pandas as pd
import sys
import os
from pathlib import Path

# src-Verzeichnis hinzufügen
sys.path.append(str(Path().resolve().parent / "src"))

# build_features.py einmalig ausführen (führt gesamte Verarbeitung aus)
os.system("python ../src/build_features.py")

# Danach Matrix laden und visualisieren
from src.visualization import visualize_feature_matrix

df = pd.read_csv("../data/feature_matrix/feature_matrix.tsv", sep="\t")
visualize_feature_matrix(df)


In [ ]:
# US14 Trainings-/Test-Split auf Gesamtdatensatz
# Split-Verhältnis: 80/20
# Seed: 42
from src.split import modified_find_tsv_file, load_tsv, split_stratified, print_label_distribution, save_splits_to_tsv, plot_label_distribution

data_dir = "../data/feature_matrix"
split_output_dir="../data/split_dataset_us14"
filepath = modified_find_tsv_file(data_dir)
df = load_tsv(filepath)
train_df, test_df = split_stratified(df)
print_label_distribution(df, train_df, test_df)
save_splits_to_tsv(train_df, test_df, split_output_dir)
plot_label_distribution(df, train_df, test_df)

In [ ]:
# Daten vorverarbeiten und bereinigen, bevor der Testdatensatz erstellt wird für US15
from pathlib import Path
import sys

# src-Verzeichnis relativ vom aktuellen Notebook-Verzeichnis
sys.path.append(str(Path().resolve().parent / "src"))

from src.new_test_data_u15.new_test_data_preprocessing import run_preprocessing_pipeline

run_preprocessing_pipeline(
    raw_data_dir="../data/new_test_data/raw_data_test/",
    cleaned_data_dir="../data/new_test_data/cleaned_data_test/",
    normalized_data_dir="../data/new_test_data/normalized_data_test/",
    time_means_folder = "../data/new_test_data/time_means/",
    labeled_data_dir="../data/new_test_data/gene_labeling_test/",
    feature_output_path="../data/new_test_data/gene_features_test/gene_features.tsv",
    build_script_path="../src/new_test_data_u15/build_features_new_test_data.py",
    visualize=False
)

In [ ]:
# US15: Definition einzelner Datensätze als Testsets
# Test - Datensätze: Yang
from src.new_test_data_u15.split_new_test_data import run_us15_split

train_dataset_dir = "../data/feature_matrix"
test_dataset_dir = "../data/new_test_data/feature_matrix_test"
us15_dataset_dir = "../data/new_test_data/split_dataset_us15"
key_column = 'id'
label_column = 'label'

run_us15_split(
        train_dir = train_dataset_dir,
        test_dir = test_dataset_dir,
        output_dir = us15_dataset_dir,
        key_column = key_column,
        label_column = label_column
)

#### US16 Vergleich verschiedener Split-Strategien
**Strategie 1**: Trainings-/Test-Split auf Gesamtdatensatz(US14)<br>
    Organismen von Gesamtdatensatz: Pseudomonas_phage_LUZ19; Pseudomonas_phage_phiKZ; Enterobacteria_phage_T7; Yersinia_phage_phiR1-37; Clostridium_phage_JD032; Vibrio_phage_VP882; Enterobacteria_phage_T4<br>
**Strategie 2**: Definition einzelner Datensätze als Testsets(US15)<br>
    Organismen von Trainingsdatensatz: Pseudomonas_phage_LUZ19; Pseudomonas_phage_phiKZ; Enterobacteria_phage_T7; Yersinia_phage_phiR1-37; Clostridium_phage_JD032; Vibrio_phage_VP882; Enterobacteria_phage_T4<br>
    Organismus von Testdatensatz: Acinetobacter_phage<br>
das angewendete Modell: **Random Forest**
- Im Skript 'src/compare_splits' wurde das Modell **Random Forest** in der Funktion 'train_and_predict' integriert. Das Skript ist so aufgebaut, dass interne Parameter des Random Forest bei Bedarf angepasst und erneut verwendet werden können.
- Verwendete Klassifikationsmetriken: Accuracy, AUC, Precision, Recall und F1-score, Support. Zusätzlich wurden die Metriken Precision, Recall und F1-score auch für jede einzelne Klasse berechnet (early, middle, late). Die letzte Zeile zeigt die Differenz zwischen den Strategien, um relative Leistungsunterschiede zu zeigen.
- Insgesamt zeigen sich keine großen Unterschiede zwischen den beiden Strategien. Strategy 2 (US15) zeigt insgesamt leicht bessere Klassifikationsmetriken, insbesondere bei AUC und F1-Score. Jedoch bietet Strategy 1 (US14) aufgrund der größeren Testmenge stabilere und zuverlässigere Ergebnisse.

In [ ]:
from src.compare_splits import (
    run_comparison,
    plot_heatmap
)
from sklearn.preprocessing import LabelEncoder
import pandas as pd

label_encoder = LabelEncoder()

results_df = run_comparison(
    "../data/split_dataset_us14/train_set.tsv",
    "../data/split_dataset_us14/test_set.tsv",
    "../data/new_test_data/split_dataset_us15/train_set.tsv",
    "../data/new_test_data/split_dataset_us15/test_set.tsv",
    label_encoder
)

pd.set_option("display.precision", 4)
print("\nVergleich der Metriken:")
print(results_df)

plot_heatmap(results_df)

#### US17: Vergleich mehrerer Klassifikator-Algorithmen
Da die bisherige Feature-Matrix ganz anders aufgebaut ist als die Embeddings aus US12, haben wir drei Modelle (SVM, Random Forest und Decision Tree) ohne Embeddings, sondern ausschließlich mit den von Hand erstellten Features trainiert.

Hauptmodell: MLP<br>
Benchmarkmodelle
- Modell 1: Decision Tree
- Modell 2: Random Forest
- Modell 3: SVM

Dataset-Split-Strategie: Trainings-/Test-Split(80/20) auf Gesamtdatensatz(US14)

In [3]:
# Decision Tree
# final model script: src/decision_tree.py
# grid search script for Decision Tree: src/grid_search_decision_tree.py
from importlib.util import spec_from_file_location, module_from_spec
import joblib
import sys

def load_model_module(path, module_name):
    """
    Lädt ein Python-Modul dynamisch aus dem gegebenen Pfad mit dem gewünschten Modulnamen.
    """
    spec = spec_from_file_location(module_name, path)
    module = module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

# Module laden
dt_module = load_model_module("../src/decision_tree.py", "decision_tree")
svc_module = load_model_module("../src/support_vector_machine.py", "svc")

# Modellpfad definieren
try:
    model_path = "../models/final_svm_model.pkl"
except Exception as e:
    print(f"modell nicht gefunden {str(e)}")

# Vortrainiertes Modell laden
final_svm_model = joblib.load(model_path)
print(type(final_svm_model))             # Sollte <class 'sklearn.svm._classes.SVC'> sein
print(hasattr(final_svm_model, "support_"))  # Sollte True sein, wenn das Modell trainiert wurde

print("Decision Tree Modell: ")
# Trainings- und Testdaten laden, Modell trainieren und auswerten
dt_module.train_and_evaluate(
    dt_module.final_model,
    "../data/split_dataset_us14/train_set.tsv",
    "../data/split_dataset_us14/test_set.tsv"
)

print("Support Vector Machine Modell:")
#svc_module.train_and_evaluate(
#    final_svm_model,
#    "../data/split_dataset_us14/train_set.tsv",
#    "../data/split_dataset_us14/test_set.tsv"
#)
svc_module.model_predict(final_svm_model, "../data/split_dataset_us14/test_set.tsv")

<class 'sklearn.svm._classes.SVC'>
True
Decision Tree Modell: 
Klassifikationsbericht (Testdaten):
              precision    recall  f1-score   support

       early       0.44      0.49      0.46       104
        late       0.41      0.42      0.42        83
      middle       0.24      0.19      0.21        64

    accuracy                           0.39       251
   macro avg       0.36      0.37      0.36       251
weighted avg       0.38      0.39      0.38       251

Support Vector Machine Modell:
              precision    recall  f1-score   support

       early       0.56      0.48      0.52       104
        late       0.41      0.37      0.39        83
      middle       0.50      0.67      0.57        64

    accuracy                           0.49       251
   macro avg       0.49      0.51      0.49       251
weighted avg       0.49      0.49      0.49       251

